In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Missing data and outliers

Real world data often needs to be cleaned before it can be used in predictive modelling. Data cleaning and preparation is incredibly important – if missing values or outliers are not handled, they can lead to the incorrect analyses of behaviour between variables.

We will discuss how to check for and fix these issues in this step, using the popular dataset `titanic.csv`. This dataset is commonly used by beginners to data science and machine learning to explore whether certain traits of passengers predicted whether they survived the Titanic sinking.

### Checking for missing values

There are many methods to check for missing values, including built-in methods in `pandas`. Let’s import  the `titanic` dataset and take a look at the statistical summary:

In [ ]:
# Import `titanic` as pandas dataframe
titanic = pd.read_csv('titanic.csv')
titanic.head()

In [ ]:
# Print shape and statistical summary of dataset
print(titanic.shape)
titanic.describe()

There are 891 rows in this dataset, but the count for the column `Age` is only 714! 

We can easily check which rows have `null` values by using `isnull()`:

In [ ]:
titanic['Age'].isnull()

This gives us a series indicating `True` if the row is `null`, and `False` if the row is `not null`. If we wanted to get the total number of `null` rows, we can do this:

In [ ]:
titanic['Age'].isnull().sum()

This checks out! 177 `null` rows and 714 `not null` rows gives us a total of 891. 

The `describe` method we used earlier only shows columns with numeric values…what if there are columns with categorical values with `null`s? We can check the entire dataframe as a whole, in case other columns also have `null` values:

In [ ]:
titanic.isnull().sum()

So now that we’ve identified missing data, how do we treat them? There are two basic approaches to handle missing values – **deletion** and **imputation**. Deleting means to delete all rows with one or more missing values. Imputation means replacing the missing values with other values based on the context of the data. Let’s go through some examples!

### Deleting missing values

You can choose to either delete an entire row or an entire column, depending on the context. In our example, if we believe `Age` is an important predictor in whether a passenger survived the Titanic shipwreck, we would definitely not want to delete the entire column. 

We may also not want to delete all the rows with missing data, since this will remove a large portion of our data (177/714 = 25% of our data!).

However, there are only 2 values missing in the column `Embarked`. We may be able to delete these two rows without decreasing our ability to create an effective predictive model too much. Let’s give it a go:

In [ ]:
# Delete any row with null values in the `Embarked` column
titanic = titanic.dropna(subset=['Embarked'], axis=0)

# Check that rows with null values in `Embarked` were dropped
titanic['Embarked'].isnull().sum()

The `dropna()` method can also be used to:
- drop entire columns (by changing `axis` to `1`),
- drop null columns or rows from the entire dataset (by not specifying `subset`), and 
- drop columns or rows where all or any values are `null` (by adding `how='all'` or `how='any'` inside the parentheses). 

### Imputing missing values

Imputation is the method of adding/replacing missing values with other values, such as `0` or the mean or median of the other `not null` values. The `fillna()` method is used to do this.

You can fill in every missing value in the entire dataset with the same value (e.g `titanic.fillna(0)` replaces all `null` with zero), or you can specify a column to fill in the missing values for just that column (e.g. `titanic[‘Ages’].fillna(0)`). 

Deciding what to fill in your values with depends on your data. Replacing `null` values in the `Ages` column with zero would not make much sense since this would make the ages of 177 people zero. Instead, let’s impute the missing values in `Ages` with the mean age of all the other passengers:

In [ ]:
# Replace any null values in `Age` with the mean age
mean_age = titanic['Age'].mean()
titanic['Age'] = titanic['Age'].fillna(mean_age)

# Check that rows with null values in `Age` were imputed
titanic['Age'].isnull().sum()

Great! There are no more missing values in `Age`! 

We can also fill in missing values with strings – let’s fill in the `null`s in `Cabin` with the string “missing”:

In [ ]:
# Replace any null values in `Cabin` with the the string "missing"
mean_age = titanic['Age'].mean()
titanic['Cabin'] = titanic['Cabin'].fillna("missing")

# Check that rows with null values in `Cabin` were imputed
titanic['Cabin'].isnull().sum()

In [ ]:
# Check dataset for null values
titanic.isnull().sum()

### Checking for outliers

Outliers are anomalous values that can distort predictive models and reduce their efficacy. Luckily they are quite easy to detect. The easiest method is to visualise the variables as **boxplots** (if you are not familiar with boxplots, [here](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51) is a great explanation).

For our `titanic` example, we used the `describe` method earlier to display statistical summaries for qualitative variables. Taking a closer look, we see that the max value for `Fare` (512.33)  is many standard deviations greater than the `mean` (32.20). This is a good indicator that there may be outliers in our data. Let’s create our boxplot to check!

In [ ]:
# Create a boxplot using matplotlib
plt.figure(figsize=(10, 5))
plt.boxplot(titanic['Fare'], vert=False)   # vert turns the plot horizontal or vertical
plt.xlabel('Fare')

plt.show()

The red line shows the mean of `Fare`. Generally speaking, points less than the first quartile (left whisker) and points higher than the third quartile (right whisker) are considered outliers.

As you see, there are quite a few values outside the third quartile, with one value in particular much higher than the rest. This could be a result of error in data collection, or some people truly did pay an exorbitant fare…either way, this outlier could distort any prediction model we try to implement. 

### Removing outliers

We may decide to remove just extreme outlier from our data, or we may remove all values higher than the third quartile. 

To remove just the extreme outlier, let’s first find out which rows these values are in:

In [ ]:
# Find the index of the rows with the max fare value
max_fare = max(titanic['Fare'])
max_fare_idx = titanic.index[titanic['Fare'] == max_fare].tolist()

print(f'Passengers in rows {max_fare_idx} paid the highest fare of ${max_fare}.')

Looks like three passengers paid 512.33 for their tickets, even though the mean fare was only 32.20! 

Now we can remove these three rows from our data.

In [ ]:
titanic2 = titanic.drop(index=max_fare_idx)
titanic2.describe()

Great! Our dataset is now 886 rows after removing these three rows with extreme `Fare` outliers. 

If we decided instead to remove all values greater than the third quartile, we can use a similar method but rather than filtering for `Fare == max(Fare)` we can filter for `Fare > Q3`:

In [ ]:
# Find the index of rows with fares greater than the third quartile
Q3 = np.percentile(titanic['Fare'], 75)
greaterthanQ3_idx = titanic.index[titanic['Fare'] > Q3].tolist()

print(f'{len(greaterthanQ3_idx)} passengers paid a fare greater than ${Q3}.')

In [ ]:
titanic3 = titanic.drop(index=greaterthanQ3_idx)
titanic3.describe()

Now, the `Fare` column has no outliers! Keep in mind, however, that this is not a fool-proof method. As you see from the statistical summary of the new dataset, there are now only 669 records – we’ve removed 220 rows! It is necessary to check whether removing all these outliers improves your linear regression model, or if removing all this data actually decreases efficacy.

Next, we will explore another issue we may run into – how to handle qualitative variables. Go back to the notebook directory in Jupyter by pressing `File` > `Open…` in the toolbar at the top, then open the notebook called `3.2 Categorical variables.ipynb`.